<a href="https://colab.research.google.com/github/mariozupan/SOMPY/blob/master/NoviClanakTXT_VAEexpanded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/NicGian/text_VAE/blob/master/text_VAE_v18.ipynb

https://nicgian.github.io/text-generation-vae/


In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
'''
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#And there you go. This allows you to access a free GPU for up to 12 hours at a time.
'''

"\nimport tensorflow as tf\ndevice_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n  raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))\n#And there you go. This allows you to access a free GPU for up to 12 hours at a time.\n"

In [3]:
#!pip install gdown
#!pip install tensorflow-gpu
#po defaultu dobijes 12GB ram, ako zelis 25 onda
'''
a = []
while(1):
    a.append(‘1’)
'''

'\na = []\nwhile(1):\n    a.append(‘1’)\n'

In [4]:
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

We set the maximum sequence length to 25, the maximun number of words in our vocabulary to 20000 and we will use 300-dimensional embeddings. Finally we load our texts from a csv. The text file is the train file of the Quora Kaggle challenge containing around 808000 sentences.

We will use pretrained Glove word embeddings as embeddings for our network. We create a matrix with one embedding for every word in our vocabulary and then we will pass this matrix as weights to the keras embedding layer of our model


VAE model

##########################################################

In [5]:

# prepare input data
def prepare_inputsMoj(X):
    X_enc = list()
    le = list()

    # label encode each column
    for i in range(X.shape[1]):
        le.append(LabelEncoder())
        le[i].fit(X[:, i])
        # encode
        _enc = le[i].transform(X[:, i])
        # store
        X_enc.append(_enc)
    return X_enc, le

In [6]:
def inverse_inputsMoj(X_enc, le):
	X_encInverse = list()
	# label encode each column
	for i in range(X_enc.shape[1]):
		# encode
		_enc = le[i].inverse_transform(X_enc[:, i])
		# store
		X_encInverse.append(_enc)
	return X_encInverse

In [7]:

#funkcija za uređenje dataseta
def urediDataset(df):

    #koristi copy kad ne zelis: After this anything you do 
    #to df2 affects only df2 and not df1 and vice versa.
    dfNew = df.copy()
    #df['epoch'] = pd.to_datetime(df['DATUM'])    
    #df['epoch'] = (df['epoch']-dt.datetime(1970,1,1)).dt.total_seconds()   

    # IZNOSkod u posebnu kolonu
    dfNew['IZNOSkod'] = dfNew[['DUGUJE','POTRAZUJE']].apply(lambda x: x['DUGUJE'] if x['DUGUJE']!=0 else x['POTRAZUJE'], axis=1)


    #skrati konto
    #df['KONTO'] = df['KONTO'].str[0:2].astype('category') #bilo je category



    #duguje potrazuje u posebunu kolonu Dkod Pkod
    dfNew['Dkod'] = dfNew[['DUGUJE']].apply(lambda x: 1 if x['DUGUJE']!=0 else 0, axis=1)
    dfNew['Pkod'] = dfNew[['POTRAZUJE']].apply(lambda x: 1 if x['POTRAZUJE']!=0 else 0, axis=1)


    # evo ga one_hot_encoded
    #df = pd.get_dummies(df, columns=["DOKUMENT", "KONTO", "DIN"], prefix=["DOKUMENT", "KONTO", "DIN"])

    #SKALIRAJ KOLONU DUGUJE i POTRAZUJR
    skaliranje = MinMaxScaler(copy=True, feature_range=(0,1))

    #df['DUGUJE'] = skaliranje.fit_transform(df[['DUGUJE']].values.reshape(-1,1))
    #df['POTRAZUJE'] = skaliranje.fit_transform(df[['POTRAZUJE']].values.reshape(-1,1))
    dfNew["IZNOSkod"]  = skaliranje.fit_transform(dfNew['IZNOSkod'].values.reshape(-1,1))

    dfNew = dfNew.reindex()

    return dfNew

In [8]:

# filename="./data/txt/dekotex2007.txt"
# dataSirovi = pd.read_csv(filename, delimiter="\t", decimal=",",
#                                                   usecols=[3,6,10,11], 
#                                                   encoding = 'utf8')

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#!ls "/content/drive/My Drive"

##then copy file(s) as needed:
#!cp "/content/drive/My Drive/xy.py" "xy.py"

##confirm that files were copied:

#!ls

In [11]:
dfVelicinaTrain=0
dfVelicinaValidate=0
dfVelicinaTest=0
dfLista = list()
for i in range(2007, 2020):
    filename = '/content/drive/My Drive/dataGL/dekotex'+str(i)+".txt"
    df = pd.read_csv(filename, delimiter="\t", decimal=",",
                                              usecols=[3,6,10,11], 
                                              encoding = 'cp1252') #cp1252 utf8
    dfLista.append(df)
    print(str(i)+". ")
    print(df.shape[0])
    
    #izmjeri velicine dataset-ova
    if (i<=2017):
      dfVelicinaTrain = dfVelicinaTrain + df.shape[0]
    elif (i==2018):
      dfVelicinaValidate = dfVelicinaValidate + df.shape[0]
    elif(i==2019):
      dfVelicinaTest = dfVelicinaTest + df.shape[0]

dataSirovi = pd.concat(dfLista)


2007. 
8011
2008. 
7469
2009. 
6660
2010. 
6813
2011. 
7103
2012. 
7413
2013. 
7049
2014. 
7115
2015. 
7540
2016. 
6921
2017. 
6544
2018. 
6121
2019. 
3864


In [12]:
print('velicina train '+str(dfVelicinaTrain))
print('velicina validate '+str(dfVelicinaValidate))
print('velicina test '+str(dfVelicinaTest))
print('velicina ukupno '+str(dfVelicinaTrain+dfVelicinaValidate+dfVelicinaTest))


velicina train 78638
velicina validate 6121
velicina test 3864
velicina ukupno 88623


In [13]:
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

In [14]:
dataNorm = urediDataset(dataSirovi)
dataNorm.head()

,DOKUMENT,KONTO,DUGUJE,POTRAZUJE,IZNOSkod,Dkod,Pkod
0,ACCOPENING,012,60383.75,0.00,0.227795,1,0
1,ACCOPENING,019,0.00,56184.56,0.225081,0,1
2,ACCOPENING,020-000,88817.76,0.00,0.246168,1,0
3,ACCOPENING,021-000,8482.09,0.00,0.194257,1,0
4,ACCOPENING,021-004,3344.86,0.00,0.190937,1,0


In [15]:
dataNorm.shape

(88623, 7)

In [16]:
dataset = dataNorm.values

X = dataset[:, [0,1,5,6]]
# format all fields as string
X = X.astype(str)

# y_train = dataset[:, [6]]
# y_train = y_train.astype(int)

In [17]:
X_enc, le = prepare_inputsMoj(X)

In [18]:
X_encProba = np.array(X_enc).transpose()

In [19]:
X_encProba = X_encProba.astype(float)
X_encProba

array([[  2.,   0.,   1.,   0.],
       [  2.,   1.,   0.,   1.],
       [  2.,   2.,   1.,   0.],
       ...,
       [ 28., 181.,   1.,   0.],
       [ 28., 182.,   0.,   1.],
       [ 28., 183.,   0.,   1.]])

In [20]:
X_encProba.size

354492

In [21]:
X_encProba[:,:].shape

(88623, 4)

In [22]:

'''
X_encProbaTrain = X_encProba[0:78638,:]#[0:44819,:]
print("2007-2017- train", X_encProbaTrain.shape)
X_encProbaValidation = X_encProba[78638:84759,:]#[44819:69303,:]
print("2018- validation", X_encProbaValidation.shape)
X_encProbaTest = X_encProba[84759:,:]
print("2019- test", X_encProbaTest.shape)

X_encProbaTrain = X_encProba[0:70000,:]#[0:44819,:]
print("2007-2017- train", X_encProbaTrain.shape)
X_encProbaValidation = X_encProba[70000:76000,:]#[44819:69303,:]
print("2018- validation", X_encProbaValidation.shape)
X_encProbaTest = X_encProba[76000:80000,:]
print("2019- test", X_encProbaTest.shape)
'''
###############################################################################################
###############################################################################################
#VRLO VAZNO I KONACNO: TRAIN I VALIDATE TREBAJU BITI OKRUGAO BROJ DJELJIV S BATCH_SIZE
###############################################################################################
###############################################################################################
dfVelicinaTrain = dfVelicinaTrain-38
X_encProbaTrain = X_encProba[0:dfVelicinaTrain,:]#[0:44819,:]
print("2007-2017- train", X_encProbaTrain.shape)
X_encProbaValidation = X_encProba[dfVelicinaTrain:dfVelicinaTrain+dfVelicinaValidate - 64,:]    #[44819:69303,:]
print("2018- validation", X_encProbaValidation.shape)
X_encProbaTest = X_encProba[dfVelicinaTrain+dfVelicinaValidate:dfVelicinaTrain+dfVelicinaValidate+dfVelicinaTest,:]
print("2019- test", X_encProbaTest.shape)

2007-2017- train (78600, 4)
2018- validation (6057, 4)
2019- test (3864, 4)


##################################################

In [23]:
#BASE_DIR = 'C:/Users/gianc/Desktop/PhD/Progetti/vae/'
#TRAIN_DATA_FILE = BASE_DIR + 'train.csv'#'train_micro.csv'
#GLOVE_EMBEDDING = BASE_DIR + 'glove.6B.300d.txt'
#VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 4 #2 #koliko trebam kolona orig 25
batch_size = 100
#NB_WORDS = NB_WORDS  #ovo je vocabular size odnosno kod mene koliko ima unikatnih, ne smije biti manji
MAX_NB_WORDS = NB_WORDS = X_encProbaTrain.shape[0] - batch_size #100000 #88623 
EMBEDDING_DIM = 50
glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM))

In [24]:
glove_embedding_matrix.shape

(78500, 50)

In [25]:
NB_WORDS

78500

In [26]:
#!pip install tensorflow==2.2.0
import tensorflow as tf
print('Morao sam downgrade na verziju 2.2.0 a aktualna je verzija 2.3.0.Dakle trenutn verzija je ' + tf.__version__)
#!pip uninstall tensorflow==1.13.1
#!pip install tensorflow_addons
#ovo je za tfa.seq2seq koji je selio s tf.contrib u posebne addons-e
import tensorflow_addons as tfa


Morao sam downgrade na verziju 2.2.0 a aktualna je verzija 2.3.0...2.2.0


In [27]:

max_len = 4 #2 #DUZINA RECENICE MAKSIMALNA, kod mene je to broj kolona
emb_dim = 50
latent_dim = 30
intermediate_dim = 80
epsilon_std = 1.0
kl_weight = 0.01
#num_sampled=8011
#act = ELU()


x = Input(shape=(max_len,))

# x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix],
#                             input_length=max_len, trainable=False)(x)

x_embed = Embedding(NB_WORDS, emb_dim,
                            input_length=max_len, trainable=False)(x)

h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2),
                  merge_mode='concat', name = 'lstmPrvi')(x_embed)

z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2, name='lstmDrugi')
decoder_mean = Dense(NB_WORDS, activation='linear')#softmax is applied in the seq2seqloss by tf #TimeDistributed()
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)


# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        #tfa je zamijenio tf.contrib u tensorflow 2.2
        xent_loss = K.sum(tfa.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)#,
                                                     #softmax_loss_function=softmax_loss_f), axis=-1)#,
        
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        xent_loss = K.mean(xent_loss)
        kl_loss = K.mean(kl_loss)
        return K.mean(xent_loss + kl_weight * kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])

opt = Adam(lr=0.001, decay=1e-6) 
vae.compile(optimizer=opt, loss=[zero_loss], metrics=[kl_loss])
vae.summary()

(None, 4) (100, 4, 78500)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 4, 50)        3925000     input_1[0][0]                    
__________________________________________________________________________________________________
lstmPrvi (Bidirectional)        (None, 160)          83840       embedding[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           4830        lstmPrvi[0][0]                   
____________________________________________________________________


# Model training

We train our model for 100 epochs through keras ".fit()". For validation data we pass the same array twice since input and labels of this model are the same. If we didn't use the "tf.contrib.seq2seq.sequence_loss" (or another similar function) we would have had to pass as labels the sequence of word one-hot encodings with dimension (batch_size, seq_len, vocab_size) consuming a lot of memory.


In [28]:
def create_model_checkpoint(dir, model_name):
    #filepath = dir + '/' + model_name + ".h5" 
    filepath = '/content/drive/My Drive/Colab Notebooks/'+dir+model_name+'.h5'
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint('data/', 'vae_seq2seq_test_very_high_std')

In [29]:

vae.fit(X_encProbaTrain, X_encProbaTrain,
     shuffle=True,
     epochs=100,
     batch_size=batch_size,
     validation_split=0.1)
     #validation_data=(X_encProbaValidation, X_encProbaValidation), callbacks=[checkpointer])


Epoch 1/100
(None, 4) (100, 4, 78500)
(None, 4) (100, 4, 78500)


_SymbolicException: ignored

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/data/'
vae.save(filename+'vae_lstmExpanded.h5')

In [ ]:

vae.load_weights(filename+'vae_lstmExpanded.h5')
#ili ovo vae_lstmExpanded koji sam snimio, to je isto
#vae.load_weights('./data/vae_lstmExpanded.h5')

In [ ]:
val_loss = vae.history.history['val_loss']
kl_loss = vae.history.history['kl_loss']
val_kl_loss = vae.history.history['val_kl_loss']
loss = vae.history.history['loss']

plt.figure(figsize=(6, 5))
plt.plot(loss)
plt.plot(val_loss)
plt.plot(kl_loss)
plt.plot(val_kl_loss)

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# plt.tight_layout()
plt.savefig(filename+"train.png")


# Kraj

In [ ]:
###########################################################################################################
from sklearn.manifold.t_sne import TSNE

#crtanje AKO THE ZEZA ERROR:
#ImportError: No module named '_tkinter', please install the python3-tk package
#ucitaj ovo:
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

# import seaborn as sns

In [ ]:

tsne = TSNE(n_components=2, random_state=0, learning_rate=500, metric='hamming',
            perplexity=40, n_iter=5000) 


x_test = X_encProbaTest

tsne_objTest = tsne.fit_transform(x_test[:,:])


tsne_dfTest = pd.DataFrame({'X':tsne_objTest[:,0], 'Y':tsne_objTest[:,1]})


In [ ]:

#########################################################################################################################

width = 40
height = 40
idAnomalije=[] # new array
plt.figure(figsize=(width,height))

y_test =  [0] * (x_test.shape[0])
y_test[0] = 1
colors = ['red','black','blue','green']

plt.scatter(tsne_objTest[:,0], tsne_objTest[:,1], s=3900, cmap=matplotlib.colors.ListedColormap(colors), alpha=0.050)


#cb = plt.colorbar()
#loc = np.arange(0,max(y_test),max(y_test)/float(len(colors)))
#cb.set_ticks(loc)
#cb.set_ticklabels(colors)



plt.savefig(filename+"tsneReal.png")

#########################################################################################################################


In [ ]:
#plt.show()
plt.close()

## Results

After training with these parameters for 100 epochs I got these results from interpolating between these two sentences:

sentence1=[‘where can i find a book on machine learning’] sentence2=[‘how can i become a successful entrepreneur’]

Generated sentences:

    where can i find a book on machine learning
    where can i find a a machine book
    how can i write a a machine book
    how can i become a successful architect
    how can i become a successful architect
    how can i become a successful entrepreneur

As we can see the results are not yet completely satisfying because not all the sentences are grammatically correct and in the interpolation the same sentence has been generated multiple times but anyway the model, even in this preliminary version seems to start working. There are certainly many improvements that could be done like:

    removing all the sentences longer than 15 instead of just truncating them
    introduce KL term annealing
    introduce the equivalent of word dropout used in the original paper for this decoder architecture
    parameter tuning (this model trains in few hours on a GTX950M with 2GB memory so it’s definitely possible to try larger nets)
    using word embeddings with higher dimensionality
    train on a more general dataset (Quora sentences are all questions)

Stay tuned for future refinings of the model!